In [2]:
import subprocess
import shutil
from itertools import product
from multiprocessing import Manager, Pool

In [3]:
# Параметры
min_value = 0
step = 1
max_value = 20

salome_executable = "~/SALOME-9.11.0-native-UB20.04-SRC/binsalome"
base_path = '.'
python_script = base_path + "/autogenerate_template.py"
core_num = 10

In [4]:
def run_simulation(shift_first_cylinder, shift_second_cylinder, shift_third_cylinder, all_data):
    new_name = f"_{shift_first_cylinder}_{shift_second_cylinder}_{shift_third_cylinder}"

    source_folder = f"{base_path}/case"
    destination_folder = f"{base_path}/case{new_name}"

    try:
        # Копирование кейса
        shutil.copytree(source_folder, destination_folder)
        print(
            f"Папка {source_folder} успешно скопирована в {destination_folder}")
        # Команда для запуска SALOME и выполнения питон-скрипта внутри SALOME
        command = f'{salome_executable} -t python {python_script} args:{shift_first_cylinder},{shift_second_cylinder},{shift_third_cylinder},{destination_folder}/mesh.unv'
        print(command)
        print("Генерация сетки")
        # Запуск команды с помощью subprocess
        subprocess.run(command, shell=True)
    except Exception as e:
        print(f"Произошла ошибка при копировании или расчете сетки: {e}")

    print("Запуск расчета")

    bash_script_path = f"./case{new_name}/Allmesh >> /dev/null"

    try:
        result = subprocess.check_output(
            bash_script_path, shell=True, text=True)
        print(result)
        print("Расчет сетки успешно выполнен.")
    except subprocess.CalledProcessError as e:
        print(f"Произошла ошибка при выполнении расчета сетки: {e}")
    except Exception as e:
        print(f"Произошла неожиданная ошибка: {e}")

    bash_script_path = f"./case{new_name}/Allrun >> /dev/null"

    try:
        result = subprocess.check_output(
            bash_script_path, shell=True, text=True)
        print(result)
        print("Расчет кейса успешно выполнен.")
        print("Ищем последнее значение")
        file_path = f"{base_path}/case{new_name}/postProcessing/heater/cellMaxHeaterT/0/volFieldValue.dat"

        # Пытаемся открыть файл
        try:
            with open(file_path, 'r') as file:
                lines = file.readlines()
                # Ищем строку с последним значением
                for line in reversed(lines):
                    if line.strip():
                        last_value = float(line.split()[-1])
                        break
                print(f"Последнее значение: {last_value:.6e}")

                # Добавляем значение в разделяемый словарь
                all_data[new_name] = last_value
        except FileNotFoundError:
            print(f"Файл '{file_path}' не найден.")
        except Exception as e:
            print(f"Произошла ошибка: {str(e)}")
    except subprocess.CalledProcessError as e:
        print(f"Произошла ошибка при выполнении расчета кейса: {e}")
    except Exception as e:
        print(f"Произошла неожиданная ошибка: {e}")

In [5]:
# Создаем разделяемый словарь для сбора результатов от каждого процесса
all_data=None
with Manager() as manager:
    all_data = manager.dict()

    # Создаем список с возможными значениями
    values = [i for i in range(min_value, max_value + step, step)]
    # Получаем все возможные комбинации с повторениями из двух чисел
    combinations_with_repeats = [(3, 11, 0), (14, 19, 0), (5, 19, 0), (17, 11, 0), (14, 10, 0)]

    # Создаем и запускаем отдельный процесс для каждой комбинации
    with Pool(core_num) as pool:
        pool.starmap(run_simulation, [(s1, s2, s3, all_data)
                     for s1, s2, s3 in combinations_with_repeats])
        pool.join

    # Выводим результаты из разделяемого словаря
    print(all_data)
    print(len(all_data))

Произошла ошибка при копировании или расчете сетки: [Errno 17] File exists: './case_3_11_0'Произошла ошибка при копировании или расчете сетки: [Errno 17] File exists: './case_14_19_0'
Произошла ошибка при копировании или расчете сетки: [Errno 17] File exists: './case_17_11_0'Произошла ошибка при копировании или расчете сетки: [Errno 17] File exists: './case_5_19_0'Запуск расчета

Произошла ошибка при копировании или расчете сетки: [Errno 17] File exists: './case_14_10_0'
Запуск расчетаЗапуск расчета


Запуск расчета
Запуск расчета




Расчет сетки успешно выполнен.

Расчет сетки успешно выполнен.

Расчет сетки успешно выполнен.

Расчет сетки успешно выполнен.


KeyboardInterrupt: 

In [7]:
# Создаем разделяемый словарь для сбора результатов от каждого процесса
all_data=None
with Manager() as manager:
    all_data = manager.dict()

    # Создаем список с возможными значениями
    values = [i for i in range(min_value, max_value + step, step)]
    # Получаем все возможные комбинации с повторениями из двух чисел
    combinations_with_repeats = [(7, 19, 0)]

    # Создаем и запускаем отдельный процесс для каждой комбинации
    with Pool(core_num) as pool:
        pool.starmap(run_simulation, [(s1, s2, s3, all_data)
                     for s1, s2, s3 in combinations_with_repeats])
        pool.join

    # Выводим результаты из разделяемого словаря
    print(all_data)
    print(len(all_data))

Папка ./case успешно скопирована в ./case_7_19_0
~/SALOME-9.11.0-native-UB20.04-SRC/binsalome -t python ./autogenerate_template.py args:7,19,0,./case_7_19_0/mesh.unv
Генерация сетки
Args: ['/home/alexesn/FOAM_project/heatExchanger/all_auto/autogenerate_template.py', '7', '19', '0', './case_7_19_0/mesh.unv']
Запуск расчета

Расчет сетки успешно выполнен.


KeyboardInterrupt: 



--> FOAM FATAL ERROR: 
cannot find file "/home/alexesn/FOAM_project/heatExchanger/all_auto/case_7_19_0/system/controlDict"

    From function virtual Foam::autoPtr<Foam::ISstream> Foam::fileOperations::uncollatedFileOperation::readStream(Foam::regIOobject&, const Foam::fileName&, const Foam::word&, bool) const
    in file global/fileOperations/uncollatedFileOperation/uncollatedFileOperation.C at line 539.

FOAM exiting

